In [1]:
import pandas as pd
import numpy as np
import re
import collections
from tqdm import tqdm
import csv
import math

In [15]:
def uniform(corpus):
    
    # Функция приводит разметку к нижнему регистру и объединяет собстенные и нарицательные существительные
    
    clear_corpus = []
    corpus.iloc[:, 2] = corpus.iloc[:, 2].str.lower()
    corpus.iloc[:, 2].replace('propn', 'noun', inplace=True)
    return corpus

In [17]:
def clearPoetry(corpus):
    
    # Функция дробит поэтический корпус по предложениям, убирает знаки препинания 
    # Помечает существительные в позиции модификатора 
    
    clear_corpus = []
    phrase = []
    file = [i for i in corpus if i[2] != 'punct']
    for i in file:
        if i[2] == 'noun' and re.search('Case=Gen', i[5]) and i[3] == 'nmod':
            i[2] = 'noun_gen'
        phrase.append(i)
    file = []
    edge = 0
    for i in range(len(phrase)):
        if not type(phrase[i][2]) is str:
            file.append(phrase[edge:i])
            edge = i+1
    return file


In [167]:
def clearProse(corpus):
    
    # Функция дробит прозаический корпус по предложениям, убирает знаки препинания 
    # Помечает существительные в позиции модификатора 
    
    clear_corpus = []
    phrase = []
    file = [i for i in corpus if i[2] != 'punct' and i[4] != 'punct']
    for i in file:
        if i[2] == 'propn':
            i[2] = 'noun'
        if i[2] == 'noun' and re.search('Case=Gen', i[5]) and i[3] == 'nmod':
            i[2] = 'noun_gen'
        phrase.append(i)
        file = []
        edge = 2
    for i in range(2, len(phrase)):
        if re.search('#', phrase[i][0]):
            file.append(phrase[edge:i])
            edge = i+1
    return file[::2]

In [116]:
def PosFreq(corpus):
    
    # Функция считает абсолютную встречаемость частей речи по всему корпусу
    
    pos_freq = {}
    for i in corpus:
        if i[2] == 'propn':
            i[2] = 'noun'
        if i[2] == 'noun' and re.search('Case=Gen', i[5]):
            i[2] = 'noun_gen'
        if not i[2] in pos_freq:
            pos_freq[(i[2])] = 1
        else:
            pos_freq[(i[2])] += 1
    return pos_freq

In [9]:
poetry_pos_freq = {'nan': 3343582,
 'noun': 2746619,
 'adj': 1226297,
 'noun_gen': 793753,
 'adv': 597106,
 'part': 465170,
 'verb': 1697025,
 'adp': 1095676,
 'punct': 3658830,
 'sconj': 227579,
 'pron': 881779,
 'cconj': 623398,
 'det': 308326,
 'num': 735557,
 'intj': 23315,
 'aux': 53711,
 'x': 18129,
 '_': 617,
 'sym': 1696, 'X' : 1, '':1}
prose_pos_freq = {'nan': 123778,
 'noun': 208481,
 'punct': 181152,
 'adp': 104064,
 'noun_gen': 104234,
 'part': 35375,
 'adj': 107800,
 'verb': 123712,
 '_': 22356,
 'cconj': 38998,
 'adv': 56113,
 'pron': 49058,
 'sconj': 20525,
 'det': 27808,
 'aux': 8145,
 'num': 18266,
 'x': 511,
 'sym': 1037,
 'intj': 100, 'X': 1}

In [21]:
def BiGrams(corpus):
    
    # Функция обходит каждое предложение и  собирает биграммы там, где между двумя словами есть синтаксическая связь
    # Предлог со связью падежа также входит в биграмму
    
    min_w, max_w, mid = ['', '', '', '', '', ''], ['', '', '', '', '', ''], ['', '', '', '', '', '']
    gr = []
    for sent in corpus:
        for word1 in sent:
            if word1[3] != 'case':
                h1 = word1[4]
                w1 = word1
                for word2 in sent:
                    if word2[0] == h1:
                        h2 = word2[4]
                        w2 = word2
                        marker = 1
                        for word3 in sent:
                            if word3 != word2 and word3 != word1 and (word3[4] == word1[0] or word3[4] == word2[0]) and word3[3] == 'case':
                                w3 = word3
                                min_w = min(int(w1[0]), int(w2[0]), int(w3[0]))
                                max_w = max(int(w1[0]), int(w2[0]), int(w3[0]))
                                for i in (w1, w2, w3):
                                    if int(i[0]) == min_w:
                                        min_w = i
                                    elif int(i[0]) == max_w:
                                        max_w = i
                                    else:
                                        mid = i
                                gr.append('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t'\
                                      .format(min_w[2], mid[2], max_w[2], min_w[3], mid[3], max_w[3],\
                                              min_w[4], mid[4], max_w[4], min_w[0], mid[0], max_w[0],\
                                              min_w[1], mid[1], max_w[1]))
                                marker = 0
                            else:
                                continue
                        if marker == 1:
                            min_w = min(int(w1[0]), int(w2[0]))
                            if int(w1[0]) == min_w:
                                min_w = w1
                                max_w = w2
                            else:
                                min_w = w2
                                max_w = w1
                            mid = ['', '', '', '', '', '']
                            gr.append('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t'\
                                      .format(min_w[2], mid[2], max_w[2], min_w[3], mid[3], max_w[3],\
                                              min_w[4], mid[4], max_w[4], min_w[0], mid[0], max_w[0],\
                                              min_w[1], mid[1], max_w[1]))
    return gr

In [177]:
def TriGrams(corpus):
    
    # Функция собирает триграммы по наличию синтаксической связи
    
    min_w, max_w, mid, mid1 = ['', '', '', '', '', ''], ['', '', '', '', '', ''], \
    ['', '', '', '', '', ''], ['', '', '', '', '', '']
    gr = []
    for sent in corpus:
        for word1 in sent:
            if word1[3] != 'case':
                h1 = word1[4]
                w1 = word1
                for word2 in sent:
                    if word2[0] == h1 and word2[3] != 'case':
                        h2 = word2[4]
                        w2 = word2
                        for word3 in sent:
                            if word3[0] == h2 and word3[3] != 'case':
                                w3 = word3
                                marker = 1
                                for word4 in sent:
                                    if word4 != word2 and word4 != word1 and word4 != word3 and \
                                    (word4[4] == word1[0] or word4[4] == word2[0] or word4[4] == word3[0])\
                                    and word4[3] == 'case':
                                        w4 = word4
                                        sorting = [int(w1[0]), int(w2[0]), int(w3[0]), int(w4[0])]
                                        sorting.sort()
                                        min_w = min(int(w1[0]), int(w2[0]), int(w3[0]), int(w4[0]))
                                        max_w = max(int(w1[0]), int(w2[0]), int(w3[0]), int(w4[0]))
                                        scales = [i for i in sorting]
                                        words = []
                                        for n in scales:
                                            for i in (w1, w2, w3, w4):
                                                if int(i[0]) == n:
                                                    words.append(i)
                                        min_w = words[0]
                                        max_w = words[3]
                                        mid = words[1]
                                        mid1 = words[2]
                                        gr.append('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t'\
                                              .format(min_w[2], mid[2], mid1[2], max_w[2], min_w[3], mid[3], mid1[3], max_w[3],\
                                                      min_w[4], mid[4], mid1[4], max_w[4], min_w[0], mid[0], mid1[0], max_w[0],\
                                                      min_w[1], mid[1], mid1[1], max_w[1]))
                                        marker = 0
                                    else:
                                        continue
                                if marker == 1:
                                    min_w = min(int(w1[0]), int(w2[0]), int(w3[0]))
                                    max_w = max(int(w1[0]), int(w2[0]), int(w3[0]))
                                    for i in (w1, w2, w3):
                                        if int(i[0]) == min_w:
                                            min_w = i
                                        elif int(i[0]) == max_w:
                                            max_w = i
                                        else:
                                            mid = i
                                        mid1 = ['', '', '', '', '', '']
                                    gr.append('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t'\
                                      .format(min_w[2], mid[2], mid1[2], max_w[2], min_w[3], mid[3], mid1[3], max_w[3],\
                                              min_w[4], mid[4], mid1[4], max_w[4], min_w[0], mid[0], mid1[0], max_w[0],\
                                              min_w[1], mid[1], mid1[1], max_w[1]))
    return gr

In [195]:
def PoetryMain():
    
    # Функция открывает поэтический корпус и собирает биграммы и триграммы
    
    poetry_corpus = pd.read_csv('Poetry.csv', delimiter='\t', low_memory=False)
    poetry_corpus = poetry_corpus[['ID_GOLD', 'WORD', 'UPOS', 'DEPREL', 'HEAD', 'FEAT']]
    poetry_corpus = uniform(poetry_corpus).values.tolist()
    poetry_sent = clearPoetry(poetry_corpus)
    bi_poetry = BiGrams(poetry_sent)
    with open('POETRY BIGRAMS.csv', 'a', encoding='utf-8') as t:
        t.write('\n'.join(bi_poetry))
    tri_poetry = TriGrams(poetry_sent)
    with open('POETRY TRIGRAMS.csv', 'a', encoding='utf-8') as t:
        t.write('\n'.join(tri_poetry))
    poetry_pos_freq = PosFreq(poetry_corpus)


In [ ]:
def ProseMain():
    
    # Функция открывает прозаический корпус и собирает биграммы и триграммы
    
    prose_corpus = pd.read_csv('Prose.csv', delimiter='\t')
    prose_corpus = prose_corpus[['ID', 'Word', 'POS', 'Role', 'Head', 'Smth']]
    prose_corpus = prose_corpus.rename(index=str, columns={'ID': 'ID_GOLD', 'Word': 'WORD', \
                                            'POS': 'UPOS', 'Role': 'DEPREL', \
                                            'Head': 'HEAD', 'Smth': 'FEAT'})
    prose_corpus = uniform(prose_corpus).values.tolist()
    prose_sent = clearProse(prose_corpus)
    bi_prose = BiGrams(poetry_sent)
    with open('PROSE BIGRAMS.csv', 'a', encoding='utf-8') as t:
        t.write('\n'.join(bi_prose))
    tri_prose = TriGrams(prose_sent)
    with open('PROSE TRIGRAMS.csv', 'a', encoding='utf-8') as t:
        t.write('\n'.join(tri_prose))
    prose_pos_freq = PosFreq(prose_corpus)

In [236]:
def CounterBI():
    
    # Функция открывает файлы с биграммами и записывает статистику
    
    # часть для прозы
    
    with open('PROSE BIGRAMS.csv', 'r', encoding='utf-8') as f:
        f = f.readlines()
    bi_grams = [' '.join(line.split('\t')[:3]) for line in f]
    counter = collections.Counter(bi_grams)
    lines = []
    for k, v in counter.items():
        k = re.sub('  ', ' X ', k)
        MI_prose = math.log(v*prose_total/\
                    (prose_pos_freq[k.split(' ')[0]]*prose_pos_freq[k.split(' ')[1]]\
                     *prose_pos_freq[k.split(' ')[2]]), 10)
        line = [k, str(v), str((v/prose_total)*100), str(MI_prose)]
        lines.append(('\t'.join(line)))
    with open('PROSE BI SCORES.csv', 'w', encoding='utf-8') as t:
        t.write('\n'.join(lines))
    
    # Часть для поэзии
            
    with open('POETRY BIGRAMS.csv', 'r', encoding='utf-8') as f:
        f = f.readlines()
    bi_grams_poetry = [' '.join(line.split('\t')[:3]) for line in f]
    counter = collections.Counter(bi_grams_poetry)
    lines = []
    for k, v in counter.items():
        k = re.sub('  ', ' X ', k)
        MI_poetry = math.log(v*poetry_total/\
                    (poetry_pos_freq[k.split(' ')[0]]*poetry_pos_freq[k.split(' ')[1]]\
                     *poetry_pos_freq[k.split(' ')[2]]), 10)
        line = [k, str(v), str((v/prose_total)*100), str(MI_poetry)]
        lines.append(('\t'.join(line)))
    with open('POETRY BI SCORES.csv', 'w', encoding='utf-8') as t:
        t.write('\n'.join(lines))

In [238]:
def CounterTRI():
    
    # Функция открывает файлы с триграммами и записывает статистику
    
    # часть для прозы
    
    with open('PROSE TRIGRAMS.csv', 'r', encoding='utf-8') as f:
        f = f.readlines()
    tri_grams = [' '.join(line.split('\t')[:4]) for line in f]
    counter = collections.Counter(tri_grams)
    lines = []
    for k, v in counter.items():
        k = re.sub('  ', ' X ', k)
        MI_prose = math.log(v*prose_total/\
                    (prose_pos_freq[k.split(' ')[0]]*prose_pos_freq[k.split(' ')[1]]\
                     *prose_pos_freq[k.split(' ')[2]]*prose_pos_freq[k.split(' ')[3]]), 10)
        line = [k, str(v), str((v/prose_total)*100), str(MI_prose)]
        lines.append(('\t'.join(line)))
    with open('PROSE TRI SCORES.csv', 'w', encoding='utf-8') as t:
        t.write('\n'.join(lines))
    
    # Часть для поэзии
            
    with open('POETRY TRIGRAMS.csv', 'r', encoding='utf-8') as f:
        f = f.readlines()
    bi_grams_poetry = [' '.join(line.split('\t')[:4]) for line in f]
    counter = collections.Counter(bi_grams_poetry)
    lines = []
    for k, v in counter.items():
        k = re.sub('  ', ' X ', k)
        try:
            MI_poetry = math.log(v*poetry_total/\
                    (poetry_pos_freq[k.split(' ')[0]]*poetry_pos_freq[k.split(' ')[1]]\
                     *poetry_pos_freq[k.split(' ')[2]]*poetry_pos_freq[k.split(' ')[3]]), 10)
            line = [k, str(v), str((v/prose_total)*100), str(MI_poetry)]
            lines.append(('\t'.join(line)))
        except IndexError:
            print(k)
    with open('POETRY TRI SCORES.csv', 'w', encoding='utf-8') as t:
        t.write('\n'.join(lines))

In [1]:
def Compare():
    
    # Функция открывает файлы с биграммами и триграммами, сопоставляет результаты и записывает в общий файл
    
    with open('POETRY BI SCORES.csv', 'r', encoding='utf-8') as f:
        poetry = f.readlines()
    with open('PROSE BI SCORES.csv', 'r', encoding='utf-8') as f:
        prose = f.readlines()
    compar = []
    for row in poetry:
        for line in prose:
            row1 = row.split('\t')[0]
            line1 = line.split('\t')[0]
            if row1 == line1:
                compar.append('{}\t{}'.format(row.strip(), '\t'.join(line.split('\t')[1:])))
    with open('BI SCORES.csv', 'w', encoding='utf-8') as t:
        t.write('\n'.join(compar))
    
    with open('POETRY TRI SCORES.csv', 'r', encoding='utf-8') as f:
        poetry = f.readlines()
    with open('PROSE TRI SCORES.csv', 'r', encoding='utf-8') as f:
        prose = f.readlines()
    compar = []
    for row in poetry:
        for line in prose:
            row1 = row.split('\t')[0]
            line1 = line.split('\t')[0]
            if row1 == line1:
                compar.append('{}\t{}'.format(row.strip(), '\t'.join(line.split('\t')[1:])))
    with open('TRI SCORES.csv', 'w', encoding='utf-8') as t:
        t.write('\n'.join(compar))

In [ ]:
def main():
    
    # Корпуса считываются из файлов, берутся нужные столбцы
    # Приводятся к общему виду с идентичными столбцами    
    # Корпуса делятся по предложениям, убираются знаки препинания

    PoetryMain()
    ProseMain()
    CounterBI()
    CounterTRI()
    Compare()

In [ ]:
if __name__== '__main__':
    main()